# Возьмем готовую разбивку номенклатур по категориям

In [1]:
import pandas
import numpy as np
import codecs
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import cross_validation, grid_search, linear_model, metrics

colnames = ['name', 'id', 'class']
data = pandas.read_csv('nameCatIdCatName.csv', names=colnames, encoding='utf8')
data.dropna(inplace = True)
names = data['name'].tolist()
classes = data['class'].tolist()

/home/hattkis/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/hattkis/.local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# todo k1s сделать выборку сбалансированной

In [3]:
x, test_data, y, test_labels = \
cross_validation.train_test_split(names,  classes, test_size = 0.3, random_state = 0)

# Решающие деревья

In [7]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
log = Pipeline([('vectorizer', TfidfVectorizer()), ('classifier', clf)])
log.fit(x, y)
score = log.score(test_data, test_labels)
print(score)

0.877943438646


# Логистическая регрессия

In [4]:
vectorizer =  TfidfVectorizer()
corpus = vectorizer.fit_transform(names)
clf = LogisticRegressionCV(n_jobs=-1, max_iter=100, refit=True)
clf.fit(corpus, classes)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [6]:
clf = LogisticRegression(C=10.0, class_weight='balanced', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           solver='lbfgs', tol=0.0001)
log = Pipeline([('vс', TfidfVectorizer()),
                      ('classifier', clf)])

log_params = {
'vc__analyzer' = ['word', 'char'],
'vc__ngram_range' = [(1, 1), (1, 2), (2, 2), (2, 3)],
'vc__use_idf' = [True, False],
'vc__smooth_idf' = [True, False] 
}
cv = cross_validation.StratifiedShuffleSplit(y, n_iter = 1, test_size = 0.2, random_state = 0)

grid_cv = grid_search.GridSearchCV(log, log_params, scoring = 'accuracy', cv = cv, n_jobs = -1)

grid_cv.fit(x, y)
print grid_cv.best_score_
print grid_cv.best_params_

Pipeline(steps=[('vectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_id...enalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))])

In [6]:
clf = LogisticRegression(C=10.0, class_weight='balanced', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           solver='lbfgs', tol=0.0001)
log = Pipeline([('vectorizer', TfidfVectorizer()),
                      ('classifier', clf)])
log.fit(x, y)

Pipeline(steps=[('vectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_id...enalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))])

In [8]:
score = log.score(test_data, test_labels)
print(score)

0.933885274985


# Полученная точность 93%

# Нейронные сети

In [5]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver = 'adam', 
                    activation = 'relu', 
                    early_stopping = True, 
                    warm_start = True,
                    validation_fraction = 0.01)

log = Pipeline([('vectorizer', TfidfVectorizer()), ('clf', clf)])

log_params = {
'clf__hidden_layer_sizes' : [(31, )]
'clf__alpha': [0.0001, 0.001, 0.1, 0.000001],
'clf__learning_rate' : ['constant', 'invscaling', 'adaptive'],
'clf__learning_rate_init' : [0.0001, 0.001, 0.1, 0.000001]
}
cv = cross_validation.StratifiedShuffleSplit(y, n_iter = 1, test_size = 0.2, random_state = 0)

grid_cv = grid_search.GridSearchCV(log, log_params, scoring = 'accuracy', cv = cv, n_jobs = -1)
grid_cv.fit(x, y)
print grid_cv.best_score_
print grid_cv.best_params_
# 0.918768216956  {'clf__hidden_layer_sizes': 31, 'clf__activation': 'relu', 'clf__solver': 'adam'}
# 0.922950196426 {'clf__hidden_layer_sizes': (800,)}

<bound method Pipeline.get_params of Pipeline(steps=[('vectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_id...      solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False))])>


In [ ]:
# from sklearn.neural_network import MLPClassifier

# clf_mlp = MLPClassifier(hidden_layer_sizes = (800, ), 
#                         activation = 'relu', 
#                         solver = 'adam', 
#                         early_stopping = True, 
#                         validation_fraction = 0.01)

# log_mlp = Pipeline([('vectorizer', TfidfVectorizer()), ('clf', clf_mlp)])
# log_mlp.fit(x, y)

# print(log_mlp.score(test_data, test_labels))

# 0.927439384979

# Будем использовать логистическую регрессию

# Раз уж мы тут все собрались, то посмотрим, что все это значит для наших  номенклатур

In [9]:
content = codecs.open("nom3utf.csv", 'r', 'utf8')
lines = [x.replace("\\", "") for x in content.readlines()]
real_test = np.random.choice(np.array(lines), size = 10)

In [11]:
predicts = log.predict_proba(real_test)
for x, predict in zip(real_test, predicts):
    print(x)
    for p, l in zip(predict, log.classes_):
        if (p > 0.3):
            print(p)
            print(l)
    print('\n')

Доксазозин табл. 4мг бл. N30 Вектор-Медика

0.674059532483
ЛЕКАРСТВА ПО РЕЦЕПТАМ


КОЛГЕЙТ з/паста Тройное действие 100мл 

0.84114776728
УХОД ЗА ПОЛОСТЬЮ РТА


Шприц Жане катетерн. 100мл Huaian City Hengchun Medical Product

0.360317053464
ЛЕКАРСТВА ПО РЕЦЕПТАМ
0.546526298699
МЕДИЦИНСКАЯ ТЕХНИКА


Вальдоксан 25мг таб п/о №14

0.877578752254
ЛЕКАРСТВА ПО РЕЦЕПТАМ


Сиалор 2% спрей для взрослых 10мл

0.578008657368
СРЕДСТВА ОТ ПРОСТУДЫ


Джес табл. п.п.о. флекс-картр. N30 Bayer Schering Pharma AG

0.336991001629
БИОЛОГИЧЕСКИ-АКТИВНЫЕ ДОБАВКИ


Черники Плоды пачка 50г (Иван Чай ЗАО)

0.743966131321
НАРОДНЫЕ СРЕДСТВА


Уролесан капс. бл. N40 Pharmaceutical Balkans

0.608698474776
ГИНЕКОЛОГИЯ И УРОЛОГИЯ


Каламин лосьон успокаивающ. 100мл Ben Shimon Floris

0.473309769353
УХОД ЗА ТЕЛОМ


Наша Мама МК бандаж Идеал до и по/род р3 белый

0.636746883582
МАМА И МАЛЫШ




# Ну, не так уж и плохо